In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [4]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2021-12-14 2021-12-15


In [5]:
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        data = TiingoExtractor.extract(ticker,start,end)
        data["ticker"] = ticker
        market.store("prices",data)
    except Exception as e:
        print(str(e))
market.disconnect()

 16%|███████████████████▏                                                                                                  | 82/505 [00:13<01:01,  6.86it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


 53%|██████████████████████████████████████████████████████████████▎                                                      | 269/505 [00:44<00:45,  5.20it/s]

project_market prices documents must be a non-empty list


 61%|██████████████████████████████████████████████████████████████████████▉                                              | 306/505 [00:50<00:38,  5.22it/s]

project_market prices documents must be a non-empty list


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 503/505 [01:23<00:00,  7.21it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment
Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [01:23<00:00,  6.03it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment
Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


In [6]:
data

In [7]:
analysis = []
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("prices",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        periods = ["year","quarter","month","week"]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|███████████████▍                                                                                                      | 66/505 [00:04<00:31, 13.95it/s]

'date'


 16%|██████████████████▋                                                                                                   | 80/505 [00:05<00:26, 15.99it/s]

'date'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [00:39<00:00, 12.81it/s]

'date'
'date'
'date'
'date'


In [8]:
a = pd.DataFrame(analysis).merge(sp5.rename(columns={"Symbol":"ticker"})[["ticker","Security","GICS Sector"]],on="ticker",how="left")
a.to_csv("analysis.csv")

In [9]:
period = "day"
a.sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector
93,0.318248,0.298176,0.166287,0.047345,0.047345,CNC,Centene Corporation,Health Care
99,-0.038854,-0.157422,-0.019410,0.026984,0.026984,CHTR,Charter Communications,Communication Services
344,1.285490,0.212698,0.131149,0.024838,0.024838,NUE,Nucor,Materials
143,0.305445,-0.091674,0.088571,0.023748,0.023748,DFS,Discover Financial Services,Financials
20,0.044174,-0.133182,0.033417,0.023198,0.023198,ALL,Allstate Corp,Financials
446,0.174901,0.028277,0.077330,0.023175,0.023175,TRV,The Travelers Companies,Financials
342,-0.037437,-0.008160,0.139812,0.022953,0.022953,NOV,Nov,Energy
78,0.365318,0.000000,0.000000,0.008613,0.022518,COG,Cabot Oil & Gas,Energy
310,0.388469,-0.080918,0.076501,0.022062,0.022062,MGM,MGM Resorts International,Consumer Discretionary
117,-0.021559,-0.147701,-0.006520,0.022008,0.022008,CMCSA,Comcast,Communication Services


In [10]:
a.groupby("GICS Sector").mean().sort_values("day_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Financials,0.351180,0.012320,0.037186,0.004388,0.004388
Consumer Staples,0.120116,0.065248,0.076444,0.000216,0.000216
Materials,0.272760,0.064041,0.046243,-0.000516,-0.000516
Communication Services,0.083506,-0.099044,0.030092,-0.002932,-0.002932
Utilities,0.149331,0.073151,0.061220,-0.004184,-0.004184
Consumer Discretionary,0.262953,0.037847,0.034379,-0.005023,-0.005023
Energy,0.557135,0.005364,0.022650,-0.006840,-0.006208
Health Care,0.182616,0.004854,0.044085,-0.006319,-0.006319
Industrials,0.252688,0.041866,0.038358,-0.010121,-0.010349
